# Measure speed for all combinations of page, screen, and scene detectors

In [1]:
from datetime import datetime

from IPython.display import display, Markdown
import pandas as pd
from video699.__main__ import PAGE_DETECTOR_NAMES, SCREEN_DETECTOR_NAMES

In [2]:
def conversion_speed(page_detector_name, screen_detector_name, scene_detector_name):
    start_time = datetime.now()
    ! python -m video699 \
        --institution 'fimu' \
        --room 'd2' \
        --camera 'default_2004' \
        --date '2019-11-12T12:00:00+01:00' \
        --documents 'Blockchain-FI-MUNI.pdf' \
        --video 'IA067-D2-20191112.mp4' \
        --output '/dev/null' \
        --page-detector '{page_detector_name}' \
        --screen-detector '{screen_detector_name}' \
        --scene-detector '{scene_detector_name}' \
        1>/dev/null 2>&1
    finish_time = datetime.now()
    conversion_duration = (finish_time - start_time).total_seconds()
    video_duration = 4545.066667
    conversion_speed = video_duration / conversion_duration
    return conversion_speed

In [3]:
def conversion_speeds(scene_detector_name):
    conversion_speeds = dict()
    for page_detector_name in PAGE_DETECTOR_NAMES:
        conversion_speeds[page_detector_name] = dict()
        for screen_detector_name in SCREEN_DETECTOR_NAMES:
            parameters = (page_detector_name, screen_detector_name, scene_detector_name)
            conversion_speeds[page_detector_name][screen_detector_name] = conversion_speed(*parameters)
    df = pd.DataFrame.from_dict(conversion_speeds)
    df = df.append(pd.Series(df.sum(), name='total'))
    df = df.T.append(pd.Series(df.T.sum(), name='total')).T
    df = df.sort_values(by='total', axis=0, ascending=False)
    df = df.sort_values(by='total', axis=1, ascending=False)
    df = df.drop('total', axis=0)
    df = df.drop('total', axis=1)
    df = df.style.background_gradient(cmap='RdYlGn', axis=None)
    df = df.format('{:.2f}×')
    display(df)

Video conversion speeds for all combinations of screen detectors (rows) and page detectors (columns) using no scene detector. The test was performed with a quad-core Intel Core i7-8550U CPU and a NVIDIA Tesla T4 GPU. Speeds are relative to video duration.

In [4]:
conversion_speeds(scene_detector_name='none')

,annotated,imagehash,siamese,vgg16
annotated,21.70×,10.31×,5.16×,2.51×
fastai,0.94×,0.89×,0.80×,0.71×


Video conversion speeds for all combinations of screen detectors (rows) and page detectors (columns) using the `FrameImageDistanceSceneDetector` scene detector. The test was performed with a quad-core Intel Core i7-8550U CPU and a NVIDIA Tesla T4 GPU. Speeds are relative to video duration.

In [5]:
conversion_speeds(scene_detector_name='distance')

,annotated,imagehash,siamese,vgg16
annotated,6.31×,4.69×,3.76×,1.99×
fastai,0.87×,0.83×,0.76×,0.65×
